# AgentCore Memory ブランチングを使用した Strands マルチエージェント並列実行システム

## はじめに

このノートブックでは、**AgentCore Memory ブランチング**を説明します。これは、共通の Memory リソースを共有しながら、複数の特化型エージェントが分離された Memory コンテキストを維持できる強力な機能です。これは、特に Strands Agent Graphs のような並列実行パターンを使用するマルチエージェントシステムに不可欠です。

## Memory ブランチングが重要な理由

マルチエージェントシステムでは、異なるエージェントがしばしば以下を必要とします：
- **別々の会話コンテキストを維持** - 各エージェントは干渉なく自分のドメインに集中
- **並列実行** - 複数のエージェントが Memory の競合なく同時に動作可能
- **共通セッションの共有** - すべてのエージェントがコンテキストを分離しながら同じユーザーセッションに貢献
- **関連履歴へのアクセス** - エージェントはコンテキストを混ぜずに自分の過去のインタラクションを取得可能

AgentCore Memory ブランチングは、コード用の Git ブランチと同様に、単一の Memory セッション内で複数の会話ブランチを許可することでこれらの課題を解決します。

## チュートリアル詳細

| 項目              | 詳細                                                                             |
|:------------------|:--------------------------------------------------------------------------------|
| チュートリアルタイプ | Memory ブランチングを備えたマルチエージェント                                      |
| エージェントユースケース | 旅行プランニングアシスタント                                                    |
| エージェントフレームワーク | Strands Agent Graph（並列実行サポート）                                      |
| LLM モデル         | Anthropic Claude Haiku 4.5                                                      |
| チュートリアルコンポーネント | AgentCore Memory ブランチング、Strands マルチエージェント Graph、並列実行     |
| 難易度             | 中級                                                                            |


学習内容：

- 異なるエージェント用の Memory ブランチを作成・管理する方法
- マルチエージェントアーキテクチャで分離された Memory コンテキストを実装する方法
- 並列実行をサポートする Strands でエージェント Graph を構築する方法
- ブランチングが安全な並行 Memory アクセスを可能にする仕組み
- ブランチ固有の会話履歴を表示・検査する方法

### シナリオコンテキスト

それぞれ独自の Memory ブランチを持つ 3 つのエージェントで**旅行プランニングシステム**を構築します：
1. **旅行コーディネーター**（main ブランチ）- 全体的な旅行プランニングを調整
2. **フライト予約アシスタント**（flight_agent_memory ブランチ）- 航空旅行のクエリを処理
3. **ホテル予約アシスタント**（hotel_agent_memory ブランチ）- 宿泊リクエストを管理

コーディネーターは、両方が必要な場合に並列実行できる特化型エージェントに委譲でき、それぞれが Memory ブランチングを通じて独自の会話履歴を維持します。

## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件
- Python 3.10 以上
- 適切な権限を持つ AWS アカウント
- AgentCore Memory に対する適切な権限を持つ AWS IAM ロール
- Amazon Bedrock モデルへのアクセス

環境をセットアップし、ブランチングサポート付きの共有 Memory リソースを作成しましょう！

## ステップ 1: 環境のセットアップ
このノートブックを動作させるために必要なすべてのライブラリをインポートし、クライアントを定義しましょう。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
from datetime import datetime
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent

Amazon Bedrock モデルと AgentCore に対する適切な権限を持つリージョンとロールを定義します

In [ ]:
import os
region = os.getenv('AWS_REGION', 'us-west-2')
MODEL_ID = "global.anthropic.claude-haiku-4-5-20251001-v1:0"

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

## ステップ 2: ブランチングサポート付き共有 Memory の作成

複数のブランチ（各エージェントに 1 つ）をサポートする単一の Memory リソースを作成します。この共有 Memory リソースは基盤として機能し、ブランチは各エージェントの会話に対して分離されたコンテキストを提供します。

Git リポジトリのように考えてください：1 つのリポジトリ（Memory リソース）に複数のブランチ（エージェントコンテキスト）があります。

In [ ]:
from bedrock_agentcore.memory import MemoryClient

In [ ]:
client = MemoryClient(region_name=region)
memory_name = "TravelAgent_STM_%s" % datetime.now().strftime("%Y%m%d%H%M%S")
memory_id = None


In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating Memory...")
    memory_name = memory_name

    # Create the memory resource
    memory = client.create_memory_and_wait(
        name=memory_name,                       # Unique name for this memory store
        description="Travel Agent STM",         # Human-readable description
        strategies=[],                          # No special memory strategies for short-term memory
        event_expiry_days=7,                    # Memories expire after 7 days
        max_wait=300,                           # Maximum time to wait for memory creation (5 minutes)
        poll_interval=10                        # Check status every 10 seconds
    )

    # Extract and print the memory ID
    memory_id = memory['id']
    print(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    print(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()

    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

### マルチエージェントシステム向け Memory ブランチングの理解

作成した Memory リソースは**ブランチング**をサポートしています。これはマルチエージェントアーキテクチャにとって重要な機能です。仕組みは以下の通りです：

**単一 Memory リソース、複数ブランチ：**
- すべてのエージェントは同じ `memory_id` と `session_id` を共有
- 各エージェントは分離されたコンテキストのために独自の `branch_name` を取得

**マルチエージェントシステムにとっての主な利点：**

1. **コンテキストの分離**：各エージェントは干渉なく独自の会話履歴を維持
   - フライトエージェントはフライト関連の会話のみを参照
   - ホテルエージェントはホテル関連の会話のみを参照
   - コーディネーターはメインのオーケストレーションフローを参照

2. **並列実行の安全性**：複数のエージェントが同時に実行可能
   - エージェントが並列実行しても Memory の競合なし
   - 各ブランチは独立してアクセス可能
   - 並行実行をサポートする Strands Agent Graphs にとって重要

3. **明確な監査証跡**：各エージェントのインタラクションが追跡可能
   - 各エージェントが何を議論したか検査
   - エージェント固有の問題をデバッグ
   - マルチエージェント会話のフローを理解

## ステップ 3: ブランチサポート付き Memory フックプロバイダーの作成

`ShortTermMemoryHook` クラスはブランチ対応の Memory 管理を実装します。これはマルチエージェントシステムで Memory ブランチングを可能にする重要なコンポーネントです。

**主な機能：**

1. **ブランチ初期化**：各エージェントのブランチを自動的に作成
   - コーディネーターエージェント用のメインブランチ
   - サブエージェント用の特化ブランチ（例：`flight_agent_memory`、`hotel_agent_memory`）
   - ブランチはメイン会話タイムラインからフォーク

2. **ブランチ固有の Memory 取得**：各エージェントは自分のコンテキストのみをロード
   - `on_agent_initialized()` はエージェントのブランチから会話履歴を取得
   - エージェント間のコンテキスト汚染を防止
   - エージェントが焦点を絞った、ドメイン固有の会話を維持できる

3. **ブランチ固有の Memory 保存**：会話は正しいブランチに保存
   - `on_message_added()` はエージェントの指定されたブランチにメッセージを保存
   - 並列エージェント実行からの同時書き込みをサポート
   - 競合状態や Memory の衝突なし

このフックプロバイダーが、AgentCore Memory で並列エージェント実行を安全かつ効率的にします。

In [ ]:
from bedrock_agentcore.memory.constants import ConversationalMessage, MessageRole
from bedrock_agentcore.memory import MemorySessionManager
class ShortTermMemoryHook(HookProvider):
    def __init__(self, memory_id: str, region_name: str = "us-west-2", branch_name: str = "main"):
        """Initialize the hook with a MemorySessionManager.

        Args:
            memory_id: The AgentCore Memory ID
            region_name: AWS region for the memory service
            branch_name: Branch name for this agent's memory (default: "main")
        """
        self.memory_manager = MemorySessionManager(
            memory_id=memory_id,
            region_name=region_name
        )
        self.memory_id = memory_id
        self.branch_name = branch_name
        self._sessions = {}  # Cache session objects per actor/session combo
        self._branch_initialized = False  # Track if branch has been created

    def _get_or_create_session(self, actor_id: str, session_id: str):
        """Get or create a MemorySession for the given actor/session.

        Args:
            actor_id: The actor identifier
            session_id: The session identifier

        Returns:
            MemorySession object
        """
        key = f"{actor_id}:{session_id}"
        if key not in self._sessions:
            self._sessions[key] = self.memory_manager.create_memory_session(
                actor_id=actor_id,
                session_id=session_id
            )
        return self._sessions[key]

    def _initialize_branch(self, actor_id: str, session_id: str):
        """Initialize a branch if it doesn't exist and this is not the main branch.

        Args:
            actor_id: The actor identifier
            session_id: The session identifier
        """
        if self._branch_initialized or self.branch_name == "main":
            return

        try:
            memory_session = self._get_or_create_session(actor_id, session_id)

            # Check if branch already exists
            branches = memory_session.list_branches()
            branch_exists = any(b.name == self.branch_name for b in branches)

            if not branch_exists:
                # Get the last event from main branch to fork from
                main_events = memory_session.list_events(branch_name="main")
                if main_events:
                    last_event = main_events[-1]
                    # Create the branch with an initial message
                    memory_session.fork_conversation(
                        root_event_id=last_event.eventId,
                        branch_name=self.branch_name,
                        messages=[
                            ConversationalMessage(f"Starting {self.branch_name} branch", MessageRole.ASSISTANT)
                        ]
                    )
                    logger.info(f"✅ Created branch: {self.branch_name}")

            self._branch_initialized = True

        except Exception as e:
            logger.error(f"Failed to initialize branch {self.branch_name}: {e}", exc_info=True)

    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when agent starts"""
        try:
            # Get session info from agent state
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")

            if not actor_id or not session_id:
                logger.warning("Missing actor_id or session_id in agent state")
                return

            # Get the memory session
            memory_session = self._get_or_create_session(actor_id, session_id)

            # For non-main branches, initialize if there are events in main branch
            if self.branch_name != "main":
                try:
                    main_events = memory_session.list_events(branch_name="main")
                    if len(main_events) > 0:
                        self._initialize_branch(actor_id, session_id)
                except Exception as e:
                    # Main branch might not exist yet on first call
                    logger.info(f"Main branch not found yet, will initialize {self.branch_name} branch later: {e}")

            # Check if the branch exists before trying to get turns
            branches = memory_session.list_branches()
            branch_exists = any(b.name == self.branch_name for b in branches)
            
            recent_turns = []
            if branch_exists:
                # Only fetch turns if branch exists
                recent_turns = memory_session.get_last_k_turns(
                    k=5,
                    branch_name=self.branch_name
                )
            else:
                logger.info(f"Branch '{self.branch_name}' does not exist yet, skipping turn retrieval")

            if len(recent_turns) > 0:
                # Format conversation history for context
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message.content.get('role', 'unknown').lower()
                        text = message.content.get('content', {}).get('text', '')
                        if text:
                            context_messages.append(f"{role.title()}: {text}")

                if context_messages:
                    context = "\n".join(context_messages)
                    logger.info(f"Loaded context from branch '{self.branch_name}' ({len(context_messages)} messages)")

                    # Add context to agent's system prompt
                    event.agent.system_prompt += (
                        f"\n\nRecent conversation history (from {self.branch_name}):\n{context}\n\n"
                        "Continue the conversation naturally based on this context."
                    )

                    logger.info(f"✅ Loaded {len(recent_turns)} recent conversation turns from branch '{self.branch_name}'")
            else:
                logger.info(f"No previous conversation history found in branch '{self.branch_name}'")

        except Exception as e:
            logger.error(f"Failed to load conversation history: {e}", exc_info=True)

    def on_message_added(self, event: MessageAddedEvent):
        """Store conversation turns in memory on the appropriate branch"""
        try:
            # Get session info from agent state
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")

            if not actor_id or not session_id:
                logger.warning("Missing actor_id or session_id in agent state")
                return

            # Get the memory session
            memory_session = self._get_or_create_session(actor_id, session_id)

            # Get the last message
            messages = event.agent.messages
            if not messages:
                return

            last_message = messages[-1]
            role_str = last_message.get("role", "").upper()
            content_text = last_message.get("content", [{}])[0].get("text", "")

            if not content_text:
                logger.debug("Skipping empty message")
                return

            # Map role string to MessageRole enum
            role_mapping = {
                "USER": MessageRole.USER,
                "ASSISTANT": MessageRole.ASSISTANT,
                "TOOL": MessageRole.TOOL,
            }
            message_role = role_mapping.get(role_str, MessageRole.USER)

            # Store the message on the appropriate branch
            if self.branch_name == "main":
                # Main branch - just add turns normally
                memory_session.add_turns(
                    messages=[ConversationalMessage(content_text, message_role)]
                )
            else:
                # Non-main branch - need to append to existing branch
                # Initialize branch if it doesn't exist
                if not self._branch_initialized:
                    self._initialize_branch(actor_id, session_id)

                # Get the latest event from this branch
                branch_events = memory_session.list_events(branch_name=self.branch_name)
                if branch_events:
                    # Add to existing branch by specifying branch name (without rootEventId)
                    memory_session.add_turns(
                        messages=[ConversationalMessage(content_text, message_role)],
                        branch={"name": self.branch_name}
                    )
                else:
                    # This shouldn't happen if _initialize_branch worked, but handle it
                    logger.warning(f"Branch {self.branch_name} not found after initialization")
                    self._initialize_branch(actor_id, session_id)

            logger.debug(f"✅ Stored message in branch '{self.branch_name}': {role_str}")

        except Exception as e:
            logger.error(f"Failed to store message: {e}", exc_info=True)

    def create_branch(self, actor_id: str, session_id: str,
                      root_event_id: str, branch_name: str,
                      messages: list):
        """Create a new conversation branch.

        Args:
            actor_id: The actor identifier
            session_id: The session identifier
            root_event_id: Event ID to branch from
            branch_name: Name for the new branch
            messages: List of ConversationalMessage objects to add to the branch
        """
        memory_session = self._get_or_create_session(actor_id, session_id)
        return memory_session.fork_conversation(
            root_event_id=root_event_id,
            branch_name=branch_name,
            messages=messages
        )

    def list_branches(self, actor_id: str, session_id: str):
        """List all branches for a session.

        Args:
            actor_id: The actor identifier
            session_id: The session identifier

        Returns:
            List of branch information
        """
        memory_session = self._get_or_create_session(actor_id, session_id)
        return memory_session.list_branches()

    def get_session(self, actor_id: str, session_id: str):
        """Get the memory session object for direct access.

        Args:
            actor_id: The actor identifier
            session_id: The session identifier

        Returns:
            MemorySession object
        """
        return self._get_or_create_session(actor_id, session_id)

    def register_hooks(self, registry: HookRegistry) -> None:
        """Register memory hooks with the registry.

        Args:
            registry: The HookRegistry to register callbacks with
        """
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## ステップ 4: Strands Agent Graph を使用したマルチエージェントアーキテクチャの作成

並列エージェント実行をサポートするフレームワークである **Strands Agent Graph** を使用してマルチエージェントシステムを構築します。各エージェントは独自の Memory ブランチで設定され、安全な並行操作を可能にします。

**アーキテクチャ概要：**
- **コーディネーターエージェント** → `main` ブランチを使用
- **フライトエージェント** → `flight_agent_memory` ブランチを使用
- **ホテルエージェント** → `hotel_agent_memory` ブランチを使用

すべてのエージェントは同じ `session_id` を共有しながら、ブランチングを通じて分離された会話コンテキストを維持します。

In [ ]:
# Import the necessary components
from strands import Agent, tool

In [ ]:
# Create unique actor IDs for each specialized agent but share the session ID
actor_id = f"travel-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"travel-session-{datetime.now().strftime('%Y%m%d%H%M%S')}"
namespace = f"travel/{actor_id}/preferences"

### ブランチ固有 Memory を持つ特化型エージェントの作成

特化型エージェント用のシステムプロンプトを定義します。各エージェントは独自の Memory ブランチで設定され、会話の分離と並列実行を可能にします。

In [ ]:
# System prompt for the hotel booking specialist
HOTEL_BOOKING_PROMPT = f"""You are a hotel booking assistant. Help customers find hotels, make reservations, and answer questions about accommodations and amenities. 
Provide clear information about availability, pricing, and booking procedures in a friendly, helpful manner."""

# System prompt for the flight booking specialist
FLIGHT_BOOKING_PROMPT = f"""You are a flight booking assistant. Help customers find flights, make reservations, and answer questions about airlines, routes, and travel policies. 
Provide clear information about flight availability, pricing, schedules, and booking procedures in a friendly, helpful manner."""

In [ ]:
flight_memory_hooks = None
hotel_memory_hooks = None

### Memory ブランチングを持つエージェントの実装

各特化型エージェントは以下で設定されます：
- 分離された Memory コンテキストのための一意の `branch_name`
- 共有セッション管理のための同じ `memory_id` と `session_id`
- ブランチ固有の操作を処理する `ShortTermMemoryHook`

**主な実装の詳細：**
- `flight_booking_agent()` はブランチ: `flight_agent_memory` を使用
- `hotel_booking_agent()` はブランチ: `hotel_agent_memory` を使用

In [ ]:
def flight_booking_agent() -> Agent:

    global flight_memory_hooks
    try:
        if flight_memory_hooks is None:
            # Create hook with branch name "flight_agent_memory"
            flight_memory_hooks = ShortTermMemoryHook(
                memory_id=memory_id,
                region_name=region,
                branch_name="flight_agent_memory"
            )

        flight_agent = Agent(
            hooks=[flight_memory_hooks],
            model=MODEL_ID,
            system_prompt=FLIGHT_BOOKING_PROMPT,
            state={"actor_id": actor_id, "session_id": session_id}
        )

        
        return flight_agent
    except Exception as e:
        return f"Error in flight booking assistant: {str(e)}"

def hotel_booking_agent() -> Agent:

    global hotel_memory_hooks
    try:
        if hotel_memory_hooks is None:
            # Create hook with branch name "hotel_agent_memory"
            hotel_memory_hooks = ShortTermMemoryHook(
                memory_id=memory_id,
                region_name=region,
                branch_name="hotel_agent_memory"
            )

        hotel_booking_agent = Agent(
            hooks=[hotel_memory_hooks],
            model=MODEL_ID,
            system_prompt=HOTEL_BOOKING_PROMPT,
            state={"actor_id": actor_id, "session_id": session_id}
        )

        return hotel_booking_agent
    except Exception as e:
        return f"Error in hotel booking assistant: {str(e)}"

### コーディネーターエージェントの作成

コーディネーターエージェントは `main` ブランチ（デフォルト）を使用し、特化型エージェントを調整します。Strands Agent Graph を使用する場合、フライトとホテルのエージェントに委譲でき、それらは並列に実行される場合があります。

In [ ]:
# System prompt for the coordinator agent
TRAVEL_AGENT_SYSTEM_PROMPT = """
You are a comprehensive travel planning assistant that coordinates between specialized tools:
- For flight-related queries (bookings, schedules, airlines, routes) → Use the flight_booking_agent
- For hotel-related queries (accommodations, amenities, reservations) → Use the hotel_booking_agent
- For complete travel packages → Use both tools as needed to provide comprehensive information
- For general travel advice or simple travel questions → Answer directly

Each agent will have its own memory in case the user asks about historic data.
When handling complex travel requests, coordinate information from both tools to create a cohesive travel plan.
Provide clear organization when presenting information from multiple sources. \
Ask max two questions per turn. Keep the messages short, don't overwhelm the customer.
"""

In [ ]:
def travel_booking_agent() -> Agent:

    agent_memory_hooks = ShortTermMemoryHook(
                    memory_id=memory_id,
                    region_name=region,
                )
    travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    hooks=[agent_memory_hooks],
    model=MODEL_ID,
    state={
        "actor_id": actor_id,
        "session_id": session_id
        }
    )

    return travel_agent

### 並列実行サポート付きエージェント Graph の構築

エージェントを **Strands Agent Graph** に組み立てます。この Graph 構造は以下を可能にします：

**並列実行：**
- コーディネーターがフライトとホテルの両方の情報を必要とする場合、両方のエージェントが同時に実行可能
- Memory ブランチングにより並行実行中の競合を防止
- 各エージェントは独立して独自のブランチに対して読み書き

**Memory ブランチマッピング：**
```
Session: travel-session-xxx
├── main ブランチ              → 旅行コーディネーター
├── flight_agent_memory      → フライト予約エージェント
└── hotel_agent_memory       → ホテル予約エージェント
```

**これが重要な理由：**
- ブランチングがないと、並列エージェントは互いの Memory を上書きしてしまう
- ブランチングにより、各エージェントは独自の会話スレッドを維持
- コーディネーターは安全に複数のエージェントに同時に委譲可能

In [ ]:
import logging
from strands import Agent
from strands.multiagent import GraphBuilder

# Enable debug logs and print them to stderr
logging.getLogger("strands.multiagent").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

# Build the Strands Agent Graph
# This graph structure enables parallel execution of specialized agents
# Memory branching ensures safe concurrent access without conflicts
builder = GraphBuilder()

# Add nodes - each agent with its own memory branch
builder.add_node(travel_booking_agent(), "travel_agent")           # Uses 'main' branch
builder.add_node(flight_booking_agent(), "flight_booking_agent")   # Uses 'flight_agent_memory' branch
builder.add_node(hotel_booking_agent(), "hotel_booking_agent")     # Uses 'hotel_agent_memory' branch

# Add edges - define which agents the coordinator can delegate to
# The graph can execute flight and hotel agents in parallel when both are needed
builder.add_edge("travel_agent", "flight_booking_agent")
builder.add_edge("travel_agent", "hotel_booking_agent")

# Set entry point - the coordinator agent receives user input first
builder.set_entry_point("travel_agent")

# Configure execution limits for safety
builder.set_execution_timeout(600)   # 10 minute timeout

# Build the graph - ready for parallel execution with isolated memory contexts
graph = builder.build()


### Memory ブランチングを備えたマルチエージェントシステムの準備完了！

エージェント Graph は以下で設定されました：
- 分離された Memory ブランチを持つ 3 つのエージェント
- Strands Agent Graph を通じた並列実行機能
- AgentCore Memory ブランチングによる安全な並行 Memory アクセス
- 自動ブランチ作成と管理

## マルチエージェントシステムのテスト

複数のエージェントをトリガーする旅行プランニングシナリオでテストしましょう：

In [ ]:
response = graph("Hello, I would like to book a trip from LA to Madrid. From July 1 to August 2.")

## Memory ブランチの検査

AgentCore Memory ブランチングの主な利点の 1 つは、各エージェントの会話履歴を独立して検査できることです。これは以下にとって重要です：

**マルチエージェントシステムのデバッグ：**
- 各エージェントが何を議論したか正確に確認
- 会話のどの部分をどのエージェントが処理したか特定
- システムを通じた情報の流れを追跡

**並列実行の理解：**
- エージェントが別々のコンテキストを維持していることを確認
- 並行実行中に Memory の競合が発生していないことを確認
- エージェントインタラクションのタイムラインを監査

会話中に作成されたブランチを調べましょう：

In [ ]:
print("\n=== Viewing Memory Branches ===")

if flight_memory_hooks or hotel_memory_hooks:
    # Get any memory session to list branches (they all point to the same session)
    hook = flight_memory_hooks if flight_memory_hooks else hotel_memory_hooks
    if hook:
        memory_session = hook.get_session(actor_id, session_id)

        # List all branches in the session
        branches = memory_session.list_branches()
        print(f"\n📊 Session has {len(branches)} branches total:")
        for branch in branches:
            print(f"  - Branch: {branch.name}")
            print(f"    └─ Events: {len(memory_session.list_events(branch_name=branch.name))}")
            print(f"    └─ Created: {branch.created}")

        print("\n💡 Each branch represents a different agent's memory:")
        print("  • 'main' = Travel coordinator conversations")
        print("  • 'flight_agent_memory' = Flight assistant conversations")
        print("  • 'hotel_agent_memory' = Hotel assistant conversations")

### ブランチ固有の会話履歴へのアクセス

各ブランチに保存された実際の会話をさらに詳しく調べましょう。これは、Memory ブランチングが共有セッションを維持しながらエージェント間の完全な分離を提供する方法を示しています。

In [ ]:
print("\n=== Accessing Branch-Specific Events ===")

if flight_memory_hooks or hotel_memory_hooks:
    hook = flight_memory_hooks if flight_memory_hooks else hotel_memory_hooks
    if hook:
        memory_session = hook.get_session(actor_id, session_id)

        # Get events from the main branch (coordinator)
        main_events = memory_session.list_events(branch_name="main")
        print(f"\n🌳 Main Branch - Coordinator ({len(main_events)} events):")
        if main_events:
            for event in main_events[-3:]:  # Show last 3 events
                for payload in event.payload:
                    if 'conversational' in payload:
                        role = payload['conversational']['role']
                        text = payload['conversational']['content']['text']
                        print(f"  {role}: {text[:100]}...")
        else:
            print("  No events found in main branch")

        # Get events from the flight agent branch
        try:
            flight_branch_events = memory_session.list_events(branch_name="flight_agent_memory")
            print(f"\n✈️  Flight Agent Branch ({len(flight_branch_events)} events):")
            if flight_branch_events:
                print("All flight-related conversations are stored here:")
                for event in flight_branch_events[-3:]:  # Show last 3 events
                    for payload in event.payload:
                        if 'conversational' in payload:
                            role = payload['conversational']['role']
                            text = payload['conversational']['content']['text']
                            print(f"  {role}: {text[:100]}...")
            else:
                print("  No events found - flight assistant wasn't called yet")
        except Exception as e:
            print(f"  Flight branch not created yet: {e}")

        # Get events from the hotel agent branch
        try:
            hotel_branch_events = memory_session.list_events(branch_name="hotel_agent_memory")
            print(f"\n🏨 Hotel Agent Branch ({len(hotel_branch_events)} events):")
            if hotel_branch_events:
                print("All hotel-related conversations are stored here:")
                for event in hotel_branch_events[-3:]:  # Show last 3 events
                    for payload in event.payload:
                        if 'conversational' in payload:
                            role = payload['conversational']['role']
                            text = payload['conversational']['content']['text']
                            print(f"  {role}: {text[:100]}...")
            else:
                print("  No events found - hotel assistant wasn't called yet")
        except Exception as e:
            print(f"  Hotel branch not created yet: {e}")

## まとめ

このノートブックでは、**AgentCore Memory ブランチング**を説明しました。これは並列実行を備えた堅牢なマルチエージェントシステムを構築するための重要な機能です：

### 主なポイント：

1. **Memory ブランチングは並列実行を可能にする**
   - 複数のエージェントが Memory の競合なく同時に実行可能
   - 各エージェントはブランチを通じて独自の会話コンテキストを維持
   - Strands Agent Graphs やその他の並列エージェントフレームワークに不可欠

2. **コンテキストの分離がエージェントのパフォーマンスを向上**
   - 特化型エージェントは干渉なく自分のドメインに集中
   - エージェント間のコンテキスト汚染なし
   - よりクリーンで関連性の高い会話

3. **分離されたコンテキストを持つ共有セッション**
   - 単一の Memory リソースとセッション ID
   - 異なるエージェント用の複数ブランチ
   - 効率的なリソース活用


### アーキテクチャパターン：

```
Memory リソース (memory_id)
  └── セッション (session_id)
      ├── main ブランチ → コーディネーターエージェント
      ├── flight_agent_memory → フライトエージェント（並列実行可能）
      └── hotel_agent_memory → ホテルエージェント（並列実行可能）
```

AgentCore Memory ブランチングにより、アプリケーションのニーズに合わせてスケールできる洗練されたマルチエージェントシステムを安全かつ効率的に構築できます。

## クリーンアップ
このノートブックで使用したリソースをクリーンアップするために Memory を削除しましょう。

In [ ]:
#client.delete_memory_and_wait(
#        memory_id = memory_id,
#        max_wait = 300,
#        poll_interval =10
#)